# Otsu's threshold in scikit-image and pyclesperanto
This notebook compares different implementations of Otsu's thresholding method. pyclesperanto's `thresholding_otsu` uses scikit-image under the hood. It just determines the histogram differently, on the GPU, and also applies the threshold on the GPU.

**Note:** benchmarking results vary heavily depending on image size, kernel size, used operations, parameters and used hardware. Use this notebook to adapt it to your use-case scenario and benchmark on your target hardware. If you have different scenarios or use-cases, you are very welcome to submit your notebook as pull-request!

In [1]:
import pyclesperanto_prototype as cle
from skimage import filters
import time

# to measure kernel execution duration properly, we need to set this flag. It will slow down exection of workflows a bit though
cle.set_wait_for_kernel_finish(True)

# selet a GPU with the following in the name. This will fallback to any other GPU if none with this name is found
cle.select_device('RTX')

<GeForce RTX 2070 on Platform: NVIDIA CUDA (1 refs)>

In [2]:
# test data
import numpy as np

test_image = np.random.random([100, 512, 512])

In [3]:
# thresholding with scikit-image
result_image = None

for i in range(0, 10):
    start_time = time.time()
    threshold = filters.threshold_otsu(test_image)
    result_image = test_image >= threshold
    print("skimage thresholding duration: " + str(time.time() - start_time))

skimage thresholding duration: 0.3397221565246582
skimage thresholding duration: 0.3506922721862793
skimage thresholding duration: 0.35404443740844727
skimage thresholding duration: 0.3540225028991699
skimage thresholding duration: 0.3640248775482178
skimage thresholding duration: 0.3759934902191162
skimage thresholding duration: 0.390505313873291
skimage thresholding duration: 0.42533421516418457
skimage thresholding duration: 0.3800032138824463
skimage thresholding duration: 0.3799574375152588


In [4]:
# thresholding with pyclesperanto
result_image = None

test_image_gpu = cle.push_zyx(test_image)

for i in range(0, 10):
    start_time = time.time()
    result_image = cle.threshold_otsu(test_image_gpu, result_image)
    print("pyclesperanto thresholding duration: " + str(time.time() - start_time))

pyclesperanto thresholding duration: 0.08271050453186035
pyclesperanto thresholding duration: 0.04976248741149902
pyclesperanto thresholding duration: 0.03821372985839844
pyclesperanto thresholding duration: 0.03829836845397949
pyclesperanto thresholding duration: 0.03852105140686035
pyclesperanto thresholding duration: 0.03886008262634277
pyclesperanto thresholding duration: 0.03933429718017578
pyclesperanto thresholding duration: 0.03995966911315918
pyclesperanto thresholding duration: 0.039511919021606445
pyclesperanto thresholding duration: 0.037897348403930664
